<h1> Proyecto de reproducción GauGAN</h1>

**Primero comenzamos descargando el código fuente de GauGAN**

descargamos algunas librerias y dependencias iniciales

In [ ]:
!pip install tqdm

In [ ]:
import numpy as np
import os 
import cv2 
from shutil import copy 
from tqdm import tqdm 
import random

In [ ]:
!git clone https://github.com/NVlabs/SPADE.git
%cd SPADE
!pip install -r requirements.txt

Ahora descarguemos el código fuente para la norma de lotes sincronizados

In [ ]:
%cd models/networks/
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

**Configuramos conjunto de datos CamVid para el entrenamiento**

Creamos una carpeta para el dataset CamVid

In [ ]:
!mkdir -p datasets/CamVid 
%cd datasets/CamVid

Descargamos el dataset de manera online o de manera local

In [ ]:
!wget -c http://web4.cs.ucl.ac.uk/staff/g.brostow/MotionSegRecData/files/701_StillsRaw_full.zip
!unzip 701_StillsRaw_full.zip

Descargamos los mapas de segmentación semántica del dataset CamVid

In [ ]:
!wget -c http://web4.cs.ucl.ac.uk/staff/g.brostow/MotionSegRecData/data/LabeledApproved_full.zip
!mkdir LabeledApproved_full 
%cd LabeledApproved_full 
!unzip ../LabeledApproved_full.zip
%cd ..

GauGAN requiere que los mapas de segmentación semántica sean imágenes con un canal, donde cada píxel representa el índice de la clase semántica a la que pertenece. Si sus mapas de segmentación semántica están en formato RGB o Polígono, entonces debe convertirlos al formato mencionado. También debe asegurarse de que los índices de clase sean números enteros continuos de 0 a n-1, donde n es el número de clases.

El conjunto de datos CamVid tiene sus mapas de segmentación semántica en formato RGB. Vamos a convertirlos al formato requerido por GauGAN. Como el de abajo.

In [ ]:
Image(filename='LabeledApproved_full/0016E5_01770_L.png')

Primero comenzamos creando el directorio para almacenar nuestras etiquetas.

In [ ]:
!mkdir GauGAN_Annotations

**Preparación de mapas de segmentación semántica**

Para convertir los mapas de segmentación, lo primero que necesitamos es un mapeo de qué color corresponde a qué clase. Esto se describe en un archivo de texto que podemos descargar del sitio web de Camvid.

---



!wget http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/data/label_colors.txt

Extraemos los colores dentro de un array numpy.

In [ ]:
with open("label_colors.txt", "r") as file:
    label_colors = file.read().split("\n")[:-1]
    label_colors = [x.split("\t") for x in label_colors]
    colors = [x[0] for x in label_colors]
    colors = [x.split(" ") for x in colors]

for i,color in enumerate(colors):
    colors[i] = [int(x) for x in color]
    
colors = np.array(colors)
print(colors)

Obtenga las direcciones de los archivos de anotaciones para trabajar.

In [ ]:
annotations_dir = "LabeledApproved_full"
annotations_files = os.listdir(annotations_dir)
annotations_files = [os.path.join(os.path.realpath("."), annotations_dir, x) for x in annotations_files]

Ahora, recorremos cada uno de estos archivos en un bucle. Luego, anidamos un bucle dentro de este bucle, que recorre cada clase de etiqueta, verifica qué píxeles pertenecen a la clase de etiqueta (verificando el valor RGB) y les asigna un índice de clase.

In [ ]:
for annotation in tqdm(annotations_files):
    # for each file
    img = cv2.imread(annotation)[:,:,::-1]
    
    h,w, _ = img.shape
    
    modified_annotation = np.zeros((h,w))
    
    for i,color in enumerate(colors):
    	# for each class color, i is the index value
        color = color.reshape(1,1,-1)
        mask = (color == img)
        
        r = mask[:,:,0]
        g = mask[:,:,1]
        b = mask[:,:,2]
        
        mask = np.logical_and(r,g)

        mask = np.logical_and(mask, b).astype(np.int64)
        
        mask *= i
        
        modified_annotation += mask
    
    save_path = annotation.replace(annotations_dir, "GauGAN_Annotations")
    cv2.imwrite(save_path, modified_annotation)

**Partición del conjunto de datos**

Primero creamos una carpeta para almacenar nuestras particiones. 

In [ ]:
!mkdir train_img train_label test_img test_label

Luego, creamos una lista de archivos que pertenecen al conjunto de entrenamiento y al conjunto de prueba.

In [ ]:
partition_percentage = 90 
annotations_dir = 'GauGAN_Annotations'
annotations_files = os.listdir(annotations_dir)
annotations_files = [os.path.join(os.path.realpath("."), annotations_dir, x) for x in annotations_files]

train_labels = random.sample(annotations_files, int(partition_percentage / 100 * len(annotations_files)))
test_labels   = [x for x in annotations_files if x not in train_labels]

train_images = [x.replace(annotations_dir, '701_StillsRaw_full').replace("_L", "") for x in train_labels]
test_images  = [x.replace(annotations_dir, '701_StillsRaw_full').replace("_L", "") for x in test_labels]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!gdown https://drive.google.com/drive/folders/15Edutj_inuColmfCZ8DWfByuT8pYQVIK

/usr/local/lib/python2.7/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=None
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/drive/folders/15Edutj_inuColmfCZ8DWfByuT8pYQVIK
To: /content/15Edutj_inuColmfCZ8DWfByuT8pYQVIK
91.4kB [00:00, 803kB/s]


In [6]:
%cd /content/drive/MyDrive/SPADE/check_points

/content/drive/MyDrive/SPADE/check_points


In [7]:
!mkdir prueba

In [4]:
!gdown https://drive.google.com/drive/folders/15Edutj_inuColmfCZ8DWfByuT8pYQVIK?usp=sharing

/usr/local/lib/python2.7/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=None
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/drive/folders/15Edutj_inuColmfCZ8DWfByuT8pYQVIK?usp=sharing
To: /content/drive/MyDrive/SPADE/CamVid/15Edutj_inuColmfCZ8DWfByuT8pYQVIK?usp=sharing
521kB [00:00, 1.01MB/s]
